# Signal processing course 2018/2019-1 @ ELTE
# Assignment 1
## 09.17.2018

## Task 9
### Random number generator with shift-register and XOR

The 128bit XorShift function (with period $2^{128}-1$) generates pseudo-random numbers with seeds $x, y, z, w$ in the following way in C language:

```c
u_int64 XorShift128(){
    t=(x^(x<<a));
    x=y;
    y=z;
    z=w;
    return w=(w^(w>>c))^(t^(t>>b))
}
```

The 160bit XorShift function (with period $2^{160}-1$), which generates pseudo-random numbers with seeds $x, y, z, w, v$, could be interpreted as follows:

```c
u_int64 XorShift160(){
    t=(x^(x>>a));
    x=y;
    y=z;
    z=w;
    w=v;
    return v=(v^(v>>c))^(t^(t>>b));
}
```

Where `<<` and `>>` operators represent left and right bitwise shifts, respectively and the `^` operator represents the bitwise XOR.

The $a, b, c$ variables are carefully selected triplets and vary between the different xorshift RNGs. In his ["Xorshift RNGs"](https://www.jstatsoft.org/article/view/v008i14) paper in 2003, George Marsaglia described *the best* triplets for some xorshift RNGs he examined. A couple relevant values for this task are listed below:

$$
[a,b,c]_{128} = \left\{ [5, 14, 1];\ [15, 4, 21];\ [23, 24, 3];\ [5, 12, 29] \right\}
$$

$$
[a,b,c]_{160} = \left\{ [2, 1, 4];\ [7, 13, 6];\ [1, 1, 20] \right\}
$$


In [ ]:
import numpy as np

import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [ ]:
# Initialize seaborn with custom settings
# Facecolor values from S. Conradi @S_Conradi/@profConradi
custom_settings = {
    'figure.facecolor': '#f4f0e8',
    'axes.facecolor': '#f4f0e8',
    'axes.edgecolor': '0.7',
    'axes.linewidth' : '2',
    'grid.color': '0.7',
    'grid.linestyle': '--',
    'grid.alpha': 0.6,
}
sns.set_theme(rc=custom_settings)

#### Number of random numbers to generate

In [ ]:
N = 5000

### XorShift128
#### Compose XorShift128 function as described above

In [ ]:
def XorShift128(x, y, z, w, a, b, c):
    '''
    Implements a xorshift RNG algorithm with a period of 2^128-1, based
    on the paper of Marsaglia "Xorshift RNGs".

    The algorithm (and the best initial values) are listed on p.4 of the
    paper.
    '''
    t = x ^ (x << a) & 0xFFFFFFFFFFFFFFFF  # Ensure 64-bit arithmetic
    x = y
    y = z
    z = w
    w = (w ^ (w >> c)) ^ (t ^ (t >> b))
    # Return updated state and a 32-bit integer
    return x, y, z, w, (w & 0xFFFFFFFF)

#### Define initial state of `XorShift128` and generate random numbers

In [ ]:
max_limit = 1e+04

# Randomly define seeds for XorShift128
x = np.random.randint(1, max_limit)
y = np.random.randint(1, max_limit)
z = np.random.randint(1, max_limit)
w = np.random.randint(1, max_limit)
# Define fine-tuned triplet for XorShift128
a, b, c = 5, 14, 1

x_128 = np.zeros(N, dtype=np.float32)
for i in range(N):
    x, y, z, w, x_128[i] = XorShift128(x=x, y=y, z=z, w=w, a=a, b=b, c=c)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

sns.kdeplot(x_128, color='0.3', fill=True, ax=ax)

ax.set_xlabel('Numbers', fontsize=14)
ax.set_ylabel('Density', fontsize=14)
ax.set_title('Fig. 1. Distribution of random numbers, by XorShift128()',
             fontsize=14, y=-0.3)

plt.show()

### XorShift160
#### Compose XorShift160 function as described above

In [ ]:
def XorShift160(x, y, z, w, v, a, b, c):
    '''
    Implements a xorshift RNG algorithm with a period of 2^160-1, based
    on the paper of Marsaglia "Xorshift RNGs".

    The algorithm (and the best initial values) are listed on p.4 of the
    paper.
    '''
    t = (x ^ (x << a)) & 0xFFFFFFFFFFFFFFFF  # Ensure 64-bit arithmetic
    x = y
    y = z
    z = w
    w = v
    v = (v ^ (v >> c)) ^ (t ^ (t >> b))
    # Return updated state and a 32-bit integer
    return x, y, z, w, v, (v & 0xFFFFFFFF)

#### Define initial state of `XorShift160` and generate random numbers

In [ ]:
max_limit = 1e+8

# Randomly define seeds for XorShift160
x = np.random.randint(1, max_limit)
y = np.random.randint(1, max_limit)
z = np.random.randint(1, max_limit)
w = np.random.randint(1, max_limit)
v = np.random.randint(1, max_limit)
# Define fine-tuned triplet for XorShift160
a, b, c = 2, 1, 4

x_160 = np.zeros(N, dtype=np.float32)
for i in range(N):
    x, y, z, w, v, x_160[i] = XorShift160(x=x, y=y, z=z, w=w, v=v, a=a, b=b, c=c)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

sns.kdeplot(x_160, color='0.3', fill=True, ax=ax)

ax.set_xlabel('Numbers', fontsize=14)
ax.set_ylabel('Density', fontsize=14)
ax.set_title('Fig. 2. Distribution of random numbers, by XorShift160()',
             fontsize=14, y=-0.3)

plt.show()

### Compare built-in RNGs
#### 1. Python's standard `random.uniform()` funtion

In [ ]:
import random

In [ ]:
x_pru = np.zeros(N, dtype=np.float32)

for i in range(N):
    x_pru[i] = random.uniform(0, 5e9)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

sns.kdeplot(x_pru, color='0.3', fill=True, ax=ax)

ax.set_xlabel('Numbers', fontsize=14)
ax.set_ylabel('Density', fontsize=14)
ax.set_title('Fig. 3. Distribution of random numbers, by random.uniform()',
             fontsize=14, y=-0.3)

plt.show()

#### 2. Numpy's `numpy.random.uniform()` function

In [ ]:
x_nru = np.random.uniform(0, 5e9, size=N)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

sns.kdeplot(x_nru, color='0.3', fill=True, ax=ax)

ax.set_xlabel('Numbers', fontsize=14)
ax.set_ylabel('Density', fontsize=14)
ax.set_title('Fig. 4. Distribution of random numbers, by numpy.random.uniform()',
             fontsize=14, y=-0.3)

plt.show()

#### 3. Numpy's `numpy.random.randint()` finction

In [ ]:
x_nri = np.random.randint(0, 5e9, size=N)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

sns.kdeplot(x_nri, color='0.3', fill=True, ax=ax)

ax.set_xlabel('Numbers', fontsize=14)
ax.set_ylabel('Density', fontsize=14)
ax.set_title('Fig. 5. Distribution of random numbers, by numpy.random.randint()',
             fontsize=14, y=-0.3)

plt.show()

#### 4. Numpy's `numpy.random.normal()` function

In [ ]:
x_nrn = np.random.normal(0, 5e9, size=N)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

sns.kdeplot(x_nrn, color='0.3', fill=True, ax=ax)

ax.set_xlabel('Numbers', fontsize=14)
ax.set_ylabel('Density', fontsize=14)
ax.set_title('Fig. 6. Distribution of random numbers, by numpy.random.randint()',
             fontsize=14, y=-0.3)

plt.show()